In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import numpy as np

from utils import tf_config

dataset="01"
memory_limit=1500
interval_eeg=10

tf_config.set_seed(seed=2)#02 [2, 3, 5, 7, 11]
tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit)

import GPyOpt

import argparse

from utils import preprocess_data, search_algorithms, train, losses_utils, metrics, eeg_utils, data_utils

from models import fmri_ae, eeg_to_fmri, uniconv_fmri

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os

from sklearn.model_selection import train_test_split, KFold

from pathlib import Path

import time

import pickle

n_individuals=getattr(data_utils, "n_individuals_"+dataset)

with tf.device('/CPU:0'):
    train_data, test_data = preprocess_data.dataset(dataset, n_individuals=n_individuals,
                                            interval_eeg=interval_eeg, 
                                            ind_volume_fit=False,
                                            standardize_fmri=True,
                                            iqr=False,
                                            verbose=True)
    eeg_train, fmri_train =train_data
    eeg_test, fmri_test = test_data

I: Starting to Load Data
I: Finished Loading Data


KeyboardInterrupt: 

In [ ]:
theta=eeg_to_fmri.parameters
#unroll hyperparameters
learning_rate=float(theta[0])
weight_decay = float(theta[1])
kernel_size = theta[2]
stride_size = theta[3]
batch_size=int(theta[4])
latent_dimension=theta[5]
n_channels=int(theta[6])
max_pool=bool(theta[7])
batch_norm=bool(theta[8])
skip_connections=bool(theta[9])
dropout=bool(theta[10])
n_stacks=int(theta[11])
outfilter=int(theta[12])
local=True

In [ ]:
with open(str(Path.home())+"/eeg_to_fmri/na_models_eeg/na_specification_2", "rb") as f:
    na_specification_eeg = pickle.load(f)
with open(str(Path.home())+"/eeg_to_fmri/na_models_fmri/na_specification_2", "rb") as f:
    na_specification_fmri = pickle.load(f)

In [ ]:
with tf.device('/CPU:0'):

    model = eeg_to_fmri.EEG_to_fMRI(latent_dimension, eeg_train.shape[1:], na_specification_eeg, n_channels,
                    weight_decay=weight_decay, skip_connections=True, batch_norm=True, fourier_features=True,
                    random_fourier=True, conditional_attention_style=True,
                    conditional_attention_style_prior=True, local=True, seed=None, 
                    fmri_args = (latent_dimension, fmri_train.shape[1:], kernel_size, stride_size, n_channels, 
                    max_pool, batch_norm, weight_decay, skip_connections,
                    n_stacks, True, False, outfilter, dropout, None, False, na_specification_fmri))
    
    model.build(eeg_train.shape, fmri_train.shape)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_fn = losses_utils.mae_cosine

    train_set = tf.data.Dataset.from_tensor_slices((eeg_train, fmri_train)).batch(batch_size)
    dev_set= tf.data.Dataset.from_tensor_slices((eeg_test, fmri_test)).batch(1)

In [ ]:
train.train(train_set, model, optimizer, loss_fn, epochs=1, u_architecture=True, val_set=None, verbose=True, verbose_batch=True)

# Making the demonstration video